In [1]:
import pandas as pd
import numpy as np
import requests
import glob 
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
# !wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

In [3]:
# !unzip source.zip

In [4]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

In [5]:
def extract_json(file):
    Dataframe = pd.read_json(file, lines= True)
    return Dataframe

In [6]:
def extract_csv(file):
    Dataframe = pd.read_csv(file)
    return Dataframe

In [7]:
def extract_xml(file):
    dataframe = pd.DataFrame(columns = ["name","height","weight"])
    tree      = ET.parse(file)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name,"height":height,"weight":weight}, ignore_index = True)
    return dataframe



In [8]:
# !pip install pymongo

In [9]:
def extract_collection(url,db,collection):
    #connect to db and get the collection db(mongoDb)
    my_columns =["name","height","weight"]
    dataframe = pd.DataFrame(columns =my_columns)
    import pymongo
    client = pymongo.MongoClient(url)
    db     = client[db]
    collection = db[collection]
    x = collection.find()
    for item in x:
        dataframe = dataframe.append(
            pd.Series([
                item['name'],
                item['height'],
                item['weight'],
                
            ]
            ,index = my_columns
        ),ignore_index = True
    )
    return dataframe

In [10]:
extract_collection("mongodb://localhost:27017/",'source','source')

,name,height,weight
0,Bienfait,170.0,60.0
1,Don,190.0,70.0
2,Nathalie,168.0,50.0


In [71]:
import mysql.connector
def connect_mysql(host,user,password,database):
    db = mysql.connector.connect(
        host = host,
        user =  user,
        password = password,
        database = database,
        buffered=True
    
    )
    return db

In [72]:
connection = connect_mysql("localhost",'root','*******','bew')

In [83]:
def extract_sql(connection,table):
    my_columns = ["id","name","email","score_id"]
    dataframe = pd.DataFrame(columns = my_columns)
    my_cursor = connection.cursor(buffered=True)
    my_cursor.execute(f"SELECT * FROM {table}")

    records = my_cursor.fetchall()
    
    for record in records:
        dataframe = dataframe.append(
            pd.Series([
                record[0],
                record[1],
                record[2],
                record[-1]
            ],index = my_columns
            ),ignore_index = True
    
        )
    return dataframe

In [84]:
extract_sql(connection,'users')

,id,name,email,score_id
0,2,Don,don@gmail.com,0
1,4,admin,admin@gmail.com,0
2,5,lion,lion@gmail.com,0
3,6,bob,bob@gmail.com,0


In [11]:
#Extract Function
def extract():
    extracted_data = pd.DataFrame(columns = ["name","height","weight"])

    for csv_file in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_csv(csv_file), ignore_index = True)

    for json_file in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_json(json_file),ignore_index=True)
    
    for xml_file in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_xml(xml_file), ignore_index = True)
    return extracted_data

In [12]:
extract().head(4)

,name,height,weight


In [66]:
# Transform¶
# The transform function does the following tasks.
# Convert height which is in inches to millimeter
# Convert weight which is in pounds to kilograms
def transform(data):
    data['height'] = round(data.height * 0.0254,2)
    data['weight'] = round(data.weight * 0.45359237,2)
    #drop duplicate also
    # data.drop_duplicates()
    #drop na
    # data.dropna()
    #LabelEncoder()
    #One-Hot Encoding
    return data

In [67]:
#Loading
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

In [68]:
#Logging
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now              =  datetime.now()
    timestamp        = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ','+ message + '\n')


In [24]:
#Running the ETL
log("ETL job Started")

In [26]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase ended")
extracted_data.head()

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


In [28]:
log("Transform phase started")
transform_data = transform(extracted_data)
log("transform phase ended")
transform_data.head(4)

,name,height,weight
0,alex,0.04,23.25
1,ajay,0.05,28.08
2,alice,0.04,31.48
3,ravi,0.04,29.28


In [32]:
log("Load phase started")
load(targetfile,transform_data)
log("Load phase Ended")

In [33]:
log("ETL Job Ended")